# 4. Восстановление функции распренделения вероятности
    1. Реализовать метод восстановления плотности вероятности двумя способами:
    1. EM-алгоритм
    2. Ядерное сглаживание
    2. Применить данные методы на любом наборе случайных точек
    3. Реализовать метод Метрополиса-Гастингса и Гибсона для несимметричного распределения. Применить два метода на основе той функции плотности, которая была восстановлена в прошлом пункте, тем самым получив изначальные точки.
    4. в методе М-Г нарисовать картинку блуждания в случае 3D-функции плотности (на доп баллы)
    5. Сравнить красный и синий набор точек (сгенерированые и исходные) (с помощью Расстояния Кульбака-Лейблера)